## TUSHAR NAGPAL -MT2022125
## MANDATE 4 : Final Submission
### This file contains code to predict the context: IPC_content given a question as mentioned in MANDATE 3.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ipc-section-wise/dataset_ipc.csv
/kaggle/input/ipc-section-wise/models.py


### Importing the .CSV dataset which was created as: IPC_Section, IPC_Context Pair

In [2]:
origDF = pd.read_csv('/kaggle/input/ipc-section-wise/dataset_ipc.csv',  encoding= 'unicode_escape')

In [3]:
!pip install -U sentence-transformers

In [4]:
origDF.drop(['Unnamed: 2'], axis = 1, inplace = True)
df = origDF.copy()
df.iloc[320]['description']

'Punishment for murder by life-convict.â\x80\x94Whoever, being under sentence of 1 [imprisonment\nfor life], commits murder, shall be punished with death.'

### Pre-processing the description to avoid some special characters affecting cosine similarity

In [5]:
import re
import nltk
nltk.download('stopwords')

import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

from collections import Counter

def preprocessingDesc(text):
#     text = re.sub(r"[^a-zA-Z0-9 ]", "", text)
    text = remove_stopwords(text)
    text_en = text.encode("ascii", "ignore")
    text = text_en.decode()
    text = text.replace("[","")
    text = text.replace("]","")
    text = text.replace("\"","")
    text = text.replace("*","")
    text = text.replace("\'","")
#     text = text.replace("\\","")
    text = text.replace("/","")
    text = text.replace("/n","")
    text = text.replace("\n","")
    text = text.replace(";","")
    text = text.lower()
    
#     cnt = Counter()
#     for word in text.split():
#         cnt[word] += 1
    
#     freqwords = set([w for(w,wc) in cnt.most_common(1)])
#     text = ' '.join([word for word in str(text).split() if word not in freqwords])
    
    return text

df['description'] = df['description'].map(lambda s:preprocessingDesc(s))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
ipcs = df['description'].tolist()

### Sentence transformer is used to create embeddings of Question and each IPC_Description to further calculate similarity between each question, [description] pair.

In [7]:
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
modelS = SentenceTransformer(model_name)

#Encoding:
sentences_embeddings_a1 = modelS.encode(ipcs)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

### If user enters any wrong spelling EDIT DISTANCE is used to correct the word.

In [13]:
question = 'punishment if woman die within seven years of marrage'
import nltk
from nltk.util import ngrams
from nltk.metrics.distance  import edit_distance

nltk.download('words')
from nltk.corpus import words
correct_words = words.words()

def autoCorrect(question):
    updated = ""
    check_words = question.split(" ")
    for word in check_words:
        temp = [(edit_distance(word, w),w) for w in correct_words if w[0]==word[0]]
        print(sorted(temp, key = lambda val:val[0])[0][1])
        updated += sorted(temp, key = lambda val:val[0])[0][1] + " "
    return updated
print(question)
question = autoCorrect(question)
print(question)
ques_emb = modelS.encode(question)

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


punishment if woman die within seven years of marrage
punishment
if
woman
die
within
seven
year
of
marriage
punishment if woman die within seven year of marriage 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
sentences_embeddings_a1.shape

(554, 768)

### Funtion that will maintain the maximum similarity index and return the index of IPC_Description which has highest cosine similarity with the question asked:

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
def predict_context(ques_emb, sentences_embeddings_a1):
    count = 0
    sim = 0
    ans = 0

    for emb in sentences_embeddings_a1:
        similar_value_a1 = cosine_similarity([ques_emb], [emb])
        if sim<similar_value_a1:
            ans = count
            sim = similar_value_a1
        count+=1

    print(ans)
    return ans
index = predict_context(ques_emb, sentences_embeddings_a1)

323


In [16]:
print(ipcs[index])
print(origDF.iloc[index]['description'])
res = origDF.iloc[index]['description']

dowry death.(1) where death woman caused burns bodily injury occurs normal circumstances seven years marriage shown soon death subjected cruelty harassment husband relative husband for, connection with, demand dowry, death shall called dowry death, husband relative shall deemed caused death. explanation.for purposes sub-section, dowry shall meaning section 2 dowry prohibition act, 1961 (28 1961). (2) whoever commits dowry death shall punished imprisonment term shall seven years extend imprisonment life.
Dowry death.â(1) Where the death of a woman is caused by any burns or bodily injury or
occurs otherwise than under normal circumstances within seven years of her marriage and it is shown that
soon before her death she was subjected to cruelty or harassment by her husband or any relative of her
husband for, or in connection with, any demand for dowry, such death shall be called âdowry deathâ, and
such husband or relative shall be deemed to have caused her death.
Explanation.âFor 

In [17]:
text_en = res.encode("ascii", "ignore")
text = text_en.decode()
answer_text = text.replace("/n","")
answer_text = text.replace("\n"," ")
answer_text

'Dowry death.(1) Where the death of a woman is caused by any burns or bodily injury or occurs otherwise than under normal circumstances within seven years of her marriage and it is shown that soon before her death she was subjected to cruelty or harassment by her husband or any relative of her husband for, or in connection with, any demand for dowry, such death shall be called dowry death, and such husband or relative shall be deemed to have caused her death. Explanation.For the purposes of this sub-section, dowry shall have the same meaning as in section 2 of the Dowry Prohibition Act, 1961 (28 of 1961). (2) Whoever commits dowry death shall be punished with imprisonment for a term which shall not be less than seven years but which may extend to imprisonment for life.]'

### This prediction will further be used in file "model-finetuned-on-IPC-dataset.ipynb"
### Model finetuned on IPC_Dataset will be given this question, answer pair and it will return the answer accordingly.